In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
df = pd.read_excel(r"C:\Users\morge\Dropbox\capstone\19092018_df_for_labeling_EXCEL_STEFAN.xlsx")

In [9]:
df.shape

(52148, 4)

In [11]:
df.columns = ['id','cat','title','abstract']

In [12]:
df['cat'] = df['cat'].astype(str).str[0:4]

In [6]:
import nltk
import random
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

porter = PorterStemmer()
wnl = WordNetLemmatizer() 
stop = stopwords.words('english')

Tokenizing each Tweet

In [13]:
from nltk.tokenize import word_tokenize

In [14]:
df['title'] = df['title'].astype(str).str.lower()

In [15]:
df['tokens'] = df['title'].apply(word_tokenize)

In [17]:
df['tokens']= df_both['tokens'].apply(lambda x:[y for y in x if y not in stop])

NameError: name 'df_both' is not defined

In [87]:
df_both['tokens']=df_both['tokens'].apply(lambda x :[y for y in x if len(y) > 1])

In [88]:
df_both['tokens'] = df_both['tokens'].apply(lambda x:[y.lower() for y in x])

In [89]:
all_words = []

for cell in df_both.tokens:
    for word in cell:
        all_words.append(word.lower())

In [90]:
all_words = [w for w in all_words if not w.startswith('/')]

In [91]:
all_words = [w for w in all_words if not w.startswith("'")]

In [92]:
len(all_words)

444096

In [93]:
all_words_freq = nltk.FreqDist(all_words)

In [94]:
all_words_freq

FreqDist({'``': 14013, 'realdonaldtrump': 8615, 'http': 6073, 'https': 5981, 'great': 4694, 'trump': 4533, 'the': 3651, 'amp': 3004, "n't": 2735, 'president': 2669, ...})

In [95]:
word_features = list(all_words_freq.keys())[:3000]

In [96]:
word_features[0:10]

['today',
 'honor',
 'honored',
 'country',
 'highest',
 'form',
 'service',
 'we',
 'owe',
 'veterans']

In [97]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [98]:
print(find_features(df_both.iloc[0,3]))

{'today': True, 'honor': True, 'honored': True, 'country': True, 'highest': True, 'form': True, 'service': True, 'we': True, 'owe': True, 'veterans': True, 'thanks': True, 'respect': True, 'freedom': True, 'progress': False, 'ballot': False, 'go': False, 'vote': False, 'make': False, 'sure': False, 'friends': False, 'family': False, 'everyone': False, 'know': False, 'votes': False, 'it': False, 'pretty': False, 'simple': False, 'find': False, 'plan': False, 'get': False, 'covered': False, 'earn': False, 'peace': False, 'mind': False, 'start': False, 'https': False, 'happened': False, 'cubs': False, 'win': False, 'world': False, 'series': False, 'that': False, 'change': False, 'even': False, 'south': False, 'sider': False, 'believe': False, 'want': False, 'come': False, 'white': False, 'house': False, 'leave': False, 'say': False, 'holy': False, 'cow': False, 'fans': False, 'sox': False, 'fan': False, 'happy': False, 'see': False, 'wrigley': False, 'rocking': False, 'last': False, 'nigh

In [99]:
df_both['feature_presence'] = df_both['tokens'].apply(find_features)

In [100]:
labeled_presidents_features = ([(tweet, 'o') for tweet in df_both.iloc[:321]['feature_presence']] + [(tweet,'t') for tweet in df_both.iloc[322:]['feature_presence']]) 

In [101]:
len(labeled_presidents_features)

37744

In [102]:
shuffled = random.shuffle(labeled_presidents_features)

In [103]:
train = labeled_presidents_features[:30000]
test = labeled_presidents_features[30001:]

In [104]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [105]:
print("Classifier accuracy:",(nltk.classify.accuracy(classifier, test))*100)

Classifier accuracy: 99.22510654784968


In [106]:
classifier.show_most_informative_features(150)

Most Informative Features
                askpotus = True                o : t      =    612.3 : 1.0
                mckinley = True                o : t      =    204.1 : 1.0
             exploration = True                o : t      =    204.1 : 1.0
                   fafsa = True                o : t      =    204.1 : 1.0
             commonsense = True                o : t      =    204.1 : 1.0
                  havana = True                o : t      =    204.1 : 1.0
                     aca = True                o : t      =    149.7 : 1.0
                   stats = True                o : t      =    122.5 : 1.0
                policing = True                o : t      =    122.5 : 1.0
                sympathy = True                o : t      =    122.5 : 1.0
                 alright = True                o : t      =    122.5 : 1.0
          astronomynight = True                o : t      =    122.5 : 1.0
                    grad = True                o : t      =    122.5 : 1.0

In [107]:
import pickle

In [108]:
with open('presidents_classifier.pickle','wb') as f:
    pickle.dump(classifier,f)

In [109]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(train)
print("MultinomialNB accuracy percent:",nltk.classify.accuracy(MNB_classifier, test))

In [ ]:
BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(train)
print("BernoulliNB accuracy percent:",nltk.classify.accuracy(BNB_classifier, test))

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(train)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, test))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(train)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, test))*100)

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(train)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, test))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(train)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, test))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(train)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, test))*100)

In [111]:
a = 'Just met John James of Michigan. He has every single quality to be your next Great Senator from Michigan. When the people of Michigan get to know John, they will say he is a true star. Also, distinguished Military and a Combat Vet!'

In [115]:
a_tokens = word_tokenize(a)

In [117]:
a_features = find_features(a_tokens)

In [118]:
classifier.classify(a_features)

't'

In [127]:
b = "Yesterday I met with high school students on Chicago’s Southwest side who spent the summer learning to code some pretty cool apps. Michelle and I are proud to support programs that invest in local youth and we’re proud of these young people."

In [128]:
b_tokens = word_tokenize(b)

In [129]:
b_features = find_features(b_tokens)

In [130]:
classifier.classify(b_features)

'o'